In [ ]:
import sys 
sys.path.append("./src")

In [1]:
from argparse import ArgumentParser
from pathlib import Path
import os
import torch
import logging
import json
import random
import numpy as np
from collections import namedtuple
from tempfile import TemporaryDirectory

from torch.utils.data import DataLoader, Dataset, RandomSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm

from pathlib import Path
import re

import time
import pandas as pd
import sklearn
from sklearn.metrics import classification_report
import  matplotlib.pyplot as plt
import scipy

import json
import math
import os
import sys
from io import open

import gzip
logger = logging.getLogger(__name__)
import  matplotlib.pyplot as plt

from collections import defaultdict
from torch import functional as F

In [2]:
from tqdm import tqdm
import glob
from multiprocessing import Pool
import time
from sklearn.linear_model import LogisticRegression
import pickle
from multiprocessing import Pool, cpu_count
import requests
import copy
import ml_metrics 

In [3]:
from bert_en import *
from evaluation import *

In [4]:
max_seq_length = 512

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
ls data/

return_val_dict.pkl  stock_price.csv  text_data_id.csv  ticker_list.csv


In [7]:
return_val_dict = pickle.load(open("data/return_val_dict.pkl", "rb"))
df_ticker = pd.read_csv("data/ticker_list.csv")
text_data_id_df = pd.read_csv("data/text_data_id.csv")
month_stock_info_df = pd.read_csv("data/stock_price.csv")

In [8]:
ticker2sector = dict(zip(df_ticker["ticker"], df_ticker["sector"]))
ticker2industry = dict(zip(df_ticker["ticker"], df_ticker["industry"]))
all_month_list = np.sort(list(set(month_stock_info_df["month"])))
month2index = dict(zip(all_month_list, range(len(all_month_list))))
text_data = dict(zip(text_data_id_df["ticker"], [
    [np.array(text_data_id_df["text_id"].iloc[i][1:-1].split(",")).astype(np.int32)] for i in range(len(text_data_id_df))]))

In [9]:
use_stock_price_list = []
sector_array = []
industry_array = []
use_text_data = []
use_ticker_list_data = []
stock_size = 59
for index, ticker in enumerate(df_ticker["ticker"]):
    if ticker in return_val_dict:
        if len(return_val_dict[ticker]) >= 12:
            if  (type(ticker2sector[ticker]) == str) :
                use_stock_price_list.append(return_val_dict[ticker][-stock_size:])
                sector_array.append(ticker2sector[ticker])
                use_text_data.append(text_data[ticker])
                industry_array.append(ticker2industry[ticker])
                use_ticker_list_data.append(ticker)

In [11]:
len(df_ticker["ticker"])

2462

In [11]:
common_similarity_mat_df = pd.read_csv("./stock_similarity_mat/common_length_mat.csv",  index_col=0)

In [12]:
use_ticker_list = np.sort(df_ticker["ticker"])

In [13]:
similarity_mat_df = pd.read_csv("./stock_similarity_mat/all_similarity_mat.csv", 
                                index_col=0)
similarity_mat_df = similarity_mat_df[df_ticker["ticker"]].T[df_ticker["ticker"]]

In [15]:
similarity_inner_mat = np.array(similarity_mat_df[
    [len(return_val_dict[ticker]) > 12 for index, ticker in enumerate(df_ticker["ticker"])]].T[
    [len(return_val_dict[ticker]) > 12 for index, ticker in enumerate(df_ticker["ticker"])]
])

In [16]:
sector2id = dict(zip(np.sort(list(set(sector_array))), range(len(set(sector_array)))))
industry2id = dict(zip(np.sort(list(set(industry_array))), range(len(set(industry_array)))))
use_label_data = [sector2id[sector] for sector in sector_array]
use_industry_data = [industry2id[sector] for sector in industry_array]

## Predict Using Bert

In [17]:
batch_size = 8
gradient_accumulation_steps = 1

In [18]:
train_data_size = 1800
valid_data_size = 262
test_data_size = (len(use_label_data) - (train_data_size + valid_data_size))

In [19]:
train_data_x = use_text_data[0:train_data_size]
train_data_y = use_label_data[0:train_data_size]
train_data_industry = use_industry_data[0:train_data_size]
train_inner_stock_mat = similarity_inner_mat[:train_data_size, :train_data_size]

In [20]:
valid_data_x = use_text_data[train_data_size:train_data_size + valid_data_size]
valid_data_y = use_label_data[train_data_size:train_data_size + valid_data_size]
valid_data_industry = use_industry_data[train_data_size:train_data_size + valid_data_size]
valid_inner_stock_mat = similarity_inner_mat[train_data_size:train_data_size + valid_data_size, 
                                                                                  train_data_size:train_data_size + valid_data_size]

In [21]:
test_data_x = use_text_data[-test_data_size:]
test_data_y = use_label_data[-test_data_size:]
test_data_industry = use_industry_data[-test_data_size:]
test_inner_stock_mat = similarity_inner_mat[-test_data_size:, -test_data_size:]

In [22]:
train_stock_label_y = np.array(use_stock_price_list)[0:train_data_size]
valid_label_y = np.array(use_stock_price_list)[train_data_size:train_data_size + valid_data_size]
test_stock_label_y = np.array(use_stock_price_list)[-test_data_size:]

In [23]:
sentence_size =    1
total_train_examples = len(train_data_x)

In [24]:
label_num = max(use_label_data)+ 1
industry_num = max(use_industry_data)+ 1

In [25]:
bert_config = BertConfig.from_pretrained('models/PreTrainigNLMbyBert/finetuned_lm/')
model_state_dict = torch.load("models/PreTrainigNLMbyBert/finetuned_lm/pytorch_model.bin")
model = BertForSequenceMeanVec(bert_config, label_num, industry_num, state_dict=model_state_dict)
device = torch.device("cuda" if torch.cuda.is_available()  else "cpu")
model.to(device)

BertForSequenceMeanVec(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
          

In [26]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        #{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         #'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

num_train_optimization_steps = int(
        total_train_examples / batch_size / gradient_accumulation_steps)
warmup_steps = 0

In [27]:
n_gpu = 2
model.to(device)
if -1 != -1:
    try:
        from apex.parallel import DistributedDataParallel as DDP
    except ImportError:
        raise ImportError(
            "Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")
    model = DDP(model)
elif n_gpu > 1:
    model = torch.nn.DataParallel(model)

In [28]:
optimizer = AdamW(optimizer_grouped_parameters)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_steps,
                                     t_total=num_train_optimization_steps)

In [29]:
sentence_size =   1
batch_size = 8
test_data_size = len(test_data_y)
max_epoch = 5

In [30]:
stock2stock_similarity_mat = copy.deepcopy(train_inner_stock_mat)

In [31]:
th_value = (stock2stock_similarity_mat[stock2stock_similarity_mat < 0.9999].mean() + 
stock2stock_similarity_mat[stock2stock_similarity_mat < 0.9999].std() * 1)

In [32]:
loss_fct = CrossEntropyLoss(ignore_index = -1)
loss_mse = MSELoss()

In [34]:
sentence_representation_all = evaluate_model_with_cosine_similarity(
    model, test_data_x, test_data_y, test_data_industry, test_data_size, 
    batch_size =  batch_size, max_seq_length = max_seq_length)

sector
same:  0.75584406
other:  0.7359424
industry
same:  0.75041085
other:  0.7386292


In [35]:
doc2soc_similarity_mat = sklearn.metrics.pairwise.cosine_similarity(sentence_representation_all)
sort_values = np.argsort(doc2soc_similarity_mat, axis = 1)[:,-1::-1]
    
print ("sector")
for top_n in [1,5,10]:
    print (top_n , ":", np.array([np.array(test_data_y)[sort_values[index]][1:top_n+1 ] ==  test_data_y[index] 
                for index in range(len(test_data_y))]).mean())

print ("industry")
for top_n in [1,5,10]:
    print (top_n , ":", np.array([np.array(test_data_industry)[sort_values[index]][1:top_n+1 ] ==  test_data_industry[index] 
    for index in range(len(test_data_industry))]).mean())

sector
1 : 0.455
5 : 0.3965
10 : 0.36825
industry
1 : 0.1575
5 : 0.111
10 : 0.08925


In [37]:
import copy
mask_volume = 1

#train_data_y_mask = np.array(copy.deepcopy(train_data_y))
#train_data_y_mask[:int(len(train_data_y_mask) * mask_volume)] = -1
#train_data_industry_mask = np.array(train_data_industry)
#train_data_industry_mask[:int(len(train_data_y_mask) * mask_volume)] = -1


seed_num = 0
train_data_y_mask = np.array(copy.deepcopy(train_data_y))
#train_data_y_mask[(train_data_y_mask % 2) != 0] = -1
# train_data_y_mask[(train_data_y_mask % 5) != 0] = -1
train_data_y_mask[(train_data_y_mask) == 0] = -1
train_data_industry_mask = np.array(train_data_industry)
#train_data_industry_mask[(train_data_y_mask % 2) != 0] = -1
# train_data_y_mask[(train_data_y_mask % 5) != 0] = -1
train_data_industry_mask[(train_data_y_mask) == 0] = -1

In [41]:
id2sector = dict(zip(range(len(set(sector_array))), np.sort(list(set(sector_array)))))
id2industry = dict(zip(range(len(set(industry_array))), np.sort(list(set(industry_array)))))

In [42]:
#test_data_industry

In [43]:
#[np.mean(np.array(test_data_industry) == industry2id[word]) for word, theme_vector_gpu in zip(theme_word_list, pooled_output)] 

In [44]:
industry_count_dict  = np.array(test_data_industry)

In [45]:
def out_actual_predicted_list(sentence_representation_all, theme_word_list, pooled_output, th_val = 0):
    actual_list = []
    predicted_list = []
    for word, theme_vector_gpu in zip(theme_word_list, pooled_output):
        theme_vector = theme_vector_gpu.detach().to("cpu").numpy()
        if np.mean(np.array(test_data_industry) == industry2id[word])  <= th_val:
            continue
        #print (word)
        doc2theme_similarity_mat = sklearn.metrics.pairwise.cosine_similarity(sentence_representation_all, theme_vector.reshape(1, -1))
        actual = np.array(range(len(test_data_industry)))[np.array(test_data_industry) == industry2id[word]]
        predicted = np.argsort(doc2theme_similarity_mat[:,0])[-1::-1]
        actual_list.append(list(actual))
        predicted_list.append(list(predicted))
    return actual_list, predicted_list 

In [46]:
theme_word_list = [word for word in industry2id]

In [47]:
theme_token_list = []
for word in theme_word_list:
    text = "[CLS] " + word +  " [SEP]"
    tokenized_text = tokenizer.tokenize(text)
    tokenized_id = tokenizer.convert_tokens_to_ids(tokenized_text)
    theme_token_list.append(tokenized_id)

max_seq_length_company = 8
input_ids_list = []
for tokenized_id in theme_token_list:
    input_array = np.zeros(max_seq_length_company, dtype=np.int)
    input_array[:min(max_seq_length_company, len(tokenized_id))] = tokenized_id[:min(max_seq_length_company, len(tokenized_id))]
    input_ids_list.append(input_array)

In [48]:
sector_token_list = []
for sector_id in np.sort(list(set(train_data_y_mask)))[1:]:
    text = "[CLS] " + id2sector[sector_id] +  " [SEP]"
    tokenized_text = tokenizer.tokenize(text)
    tokenized_id = tokenizer.convert_tokens_to_ids(tokenized_text)
    sector_token_list.append(tokenized_id)

max_seq_length_company = 8
input_sector_ids_list = []
for tokenized_id in sector_token_list:
    input_array = np.zeros(max_seq_length_company, dtype=np.int)
    input_array[:min(max_seq_length_company, len(tokenized_id))] = tokenized_id[:min(max_seq_length_company, len(tokenized_id))]
    input_sector_ids_list.append(input_array)

In [51]:
cos = nn.CosineSimilarity(dim=2, eps=1e-6)

In [64]:
model.module.to("cuda")

BertForSequenceMeanVec(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
          

In [66]:
write_file_name = "test"

In [ ]:
def train(max_epoch, write_file_name, use_loss_sector = True, use_loss_stock = True, use_sector_nm_loss = True, use_loss_industry = False):
    stock_loss_scale = 0
    max_valid_f1_score = 0
    for epoch in range(max_epoch):
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        global_step = 0
        first_time = time.time()
        model.train()
        random_perm = np.random.permutation(train_data_size)
        train_data_x_rand = np.array(train_data_x[0:train_data_size])[random_perm]
        train_data_y_rand = np.array(train_data_y_mask[:train_data_size])[random_perm]
        train_data_y_rand_stock = np.array(train_stock_label_y[0:train_data_size])[random_perm]
        train_data_y_rand_industry = np.array(train_data_industry_mask[0:train_data_size])[random_perm]
        stock_similarity_random = train_inner_stock_mat[random_perm].T[random_perm]
        #for data_index, (doc, label) in enumerate(zip(train_data_x_rand, train_data_y_rand)):
        for data_index in range(0, len(train_data_x_rand), batch_size):
            loss = 0
            data_batch = train_data_x_rand[data_index:data_index+batch_size]
            doc_batch = [doc[0] for doc in data_batch]
            #if  len(doc) == 0:
                #continue
            #print ((time.time()-first_time), (", "),  (data_index))
            logits = 0
            stock_logits = 0
            #for doc_index in range(0, min(len(doc), sentence_size), batch_size):
            label_batch = np.array(train_data_y_rand[data_index:data_index+batch_size])
            #label_batch_stock = train_data_y_rand_stock[data_index:data_index+batch_size]
            label_batch_industry = train_data_y_rand_industry[data_index:data_index+batch_size]
            input_array_doc = []
            for doc_batch_index, input_ids in enumerate(doc_batch):
                    input_array = np.zeros(max_seq_length, dtype=np.int)
                    input_array[:min(max_seq_length, len(input_ids))] = input_ids[:min(max_seq_length, len(input_ids))]
                    input_array_doc.append(input_array)
            input_ids = torch.LongTensor(np.array(input_array_doc).astype(np.int32))
            label_logits, pooled_output, industry_logits = model(input_ids, 
                                                                                       labels= torch.LongTensor(label_batch), 
                                                                                       label_industry = label_batch_industry,                                                   
                                                                                       stock_vol = 0)


            logits  = label_logits
            industry_logits = industry_logits
            stock_similarity = torch.Tensor(stock_similarity_random[data_index:data_index+batch_size, data_index:data_index+batch_size])
            sector_similarity = torch.Tensor(np.array(train_data_y_rand[data_index:data_index+batch_size][:,np.newaxis])
                                                       == np.array(train_data_y_rand[data_index:data_index+batch_size][np.newaxis, :]))

            pooled_output_norm = (pooled_output.T/F.norm(pooled_output, dim = 1)).T
            doc2doc_similarity =  torch.matmul(pooled_output_norm, pooled_output_norm.T)
            loss_stock = loss_mse(doc2doc_similarity[stock_similarity != 0],  stock_similarity[stock_similarity != 0].to("cuda"))
            loss_sector = loss_fct(logits,  torch.LongTensor(np.array(train_data_y_rand[data_index:data_index+batch_size])).to("cuda"))
            loss_industry = loss_fct(industry_logits, torch.LongTensor(np.array(train_data_y_rand_industry[data_index:data_index+batch_size])).to("cuda"))

            #loss = outputs[0].mean()
            tr_loss += loss_sector.detach().to("cpu").item()
            
            if use_loss_sector:
                loss += loss_sector /(32/batch_size)
            if use_loss_industry:
                loss += loss_industry/(32/batch_size)
            if use_loss_stock:
                loss += loss_stock/(32/batch_size)
            
            
            if use_sector_nm_loss:
                if ((((data_index + batch_size) % 32) == 0) or (data_index == (len(train_data_x_rand)-1))):
                    input_ids = torch.LongTensor(np.array(input_sector_ids_list).astype(np.int32))
                    label_sector_logits, pooled_sector_output = model(input_ids, labels= torch.LongTensor(np.sort(list(set(train_data_y_mask)))[1:]),  
                                                            label_industry = None,
                                                            labels_stock =  None, stock_vol = 0)

                    loss_regularize = loss_fct(label_sector_logits,  torch.LongTensor(np.array(np.sort(list(set(train_data_y_mask)))[1:])).to("cuda"))
                    loss += 0.1 * loss_regularize
                    
            loss.backward()
            if ((((data_index + batch_size) % 32) == 0) or (data_index == (len(train_data_x_rand)-1))):
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1


        pred_label_valid, answer_label_valid, pred_industry_valid, answer_indesutry_valid, tr_loss_valid = evaluate_model(
        model, valid_data_x, valid_data_y, valid_data_industry, valid_data_size)
        valid_f1_score = sklearn.metrics.f1_score(answer_label_valid, pred_label_valid, average = "macro")
        if valid_f1_score >= max_valid_f1_score:
            print ("epoch" + str(epoch ) +": " + str(tr_loss_valid/valid_data_size), valid_f1_score, " time:", str(time.time()-first_time))
            max_valid_f1_score = valid_f1_score
            model.module.to("cpu").save_pretrained(write_file_name)
            model.module.to("cuda")

In [67]:
train(max_epoch, write_file_name, use_loss_sector = True, use_loss_stock = True, use_sector_nm_loss = True)

epoch0: 0.32226130434574973 0.21278984441848328  time: 91.81227731704712
epoch3: 0.35246708602395677 0.21288459410525137  time: 282.90406680107117
epoch4: 0.35190293079114143 0.2607106336311703  time: 291.11462235450745
